In [48]:
%reload_ext autoreload
%autoreload 2

[autoreload of transpile_benchy.interface failed: Traceback (most recent call last):
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autore

### Example benchmarking comparison between 2 Qiskit transpilers

In [54]:
from transpile_benchy.interface import QASMBench
import qiskit

qasm_bench = QASMBench(size="small", filter_list=["toffoli"])
all_circuits = (
    qasm_bench.get_all_possible_circuits()
)  # list of names of all possible circuits for qasm

from transpile_benchy.metrics import DepthMetric
from qiskit.circuit.library.standard_gates.x import CXGate, XGate

basis_gate = CXGate()

depth_metric = DepthMetric(basis_gate=basis_gate)

from qiskit.transpiler import CouplingMap

coupling_map = CouplingMap.from_grid(4, 5)

[autoreload of transpile_benchy.interface failed: Traceback (most recent call last):
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/jones1/transpile_benchy/.venv/lib/python3.9/site-packages/IPython/extensions/autore

In [55]:
all_circuits

['adder_n10',
 'variational_n4',
 'hs4_n4',
 'deutsch_n2',
 'shor_n5',
 'teleportation_n3',
 'qec_en_n5',
 'simon_n6',
 'qft_n4',
 'basis_change_n3',
 'grover_n2',
 'error_correctiond3_n5',
 'adder_n4',
 'ipea_n2',
 'hhl_n10',
 'qaoa_n3',
 'quantumwalks_n2',
 'qaoa_n6',
 'hhl_n7',
 'pea_n5',
 'ising_n10',
 'dnn_n2',
 'dnn_n8',
 'iswap_n2',
 'wstate_n3',
 'lpn_n5',
 'cat_state_n4',
 'toffoli_n3',
 'bell_n4',
 'qec_sm_n5',
 'linearsolver_n3',
 'basis_test_n4',
 'basis_trotter_n4',
 'fredkin_n3',
 'qpe_n9',
 'qrng_n4',
 'bb84_n8']

In [50]:
from transpile_benchy.runner import CustomPassManager
from qiskit.transpiler.passes import (
    DenseLayout,
    LookaheadSwap,
    TrivialLayout,
    BasicSwap,
    SabreLayout,
    SabreSwap,
)
from qiskit.transpiler.passes import (
    ApplyLayout,
    SetLayout,
    FullAncillaAllocation,
    EnlargeWithAncilla,
)


class Trivial_Basic(CustomPassManager):
    def main_process(self):
        """Process the circuit."""
        self.pm.append(
            [
                TrivialLayout(self.coupling),
                FullAncillaAllocation(self.coupling),
                EnlargeWithAncilla(),
                ApplyLayout(),
                BasicSwap(self.coupling),
            ]
        )


# class Dense_Lookahead(CustomPassManager):
#     def main_process(self, circuit):
#         """Process the circuit."""
#         self.main_pm.append(
#             [
#                 DenseLayout(coupling_map),
#                 FullAncillaAllocation(coupling_map),
#                 EnlargeWithAncilla(),
#                 ApplyLayout(),
#                 LookaheadSwap(coupling_map),
#             ]
#         )
#         return self.main_pm.run(circuit)


class SABRE(CustomPassManager):
    def main_process(self):
        """Process the circuit."""
        self.pm.append(
            [
                SabreLayout(self.coupling),
                # FullAncillaAllocation(coupling_map),
                # EnlargeWithAncilla(),
                # ApplyLayout(),
                SabreSwap(self.coupling),
            ]
        )

In [51]:
import logging
from transpile_benchy.benchmark import Benchmark

# Create a specific logger for transpile_benchy and set its level to DEBUG
transpile_benchy_logger = logging.getLogger("transpile_benchy")
# transpile_benchy_logger.setLevel(logging.DEBUG)
transpile_benchy_logger.setLevel(logging.INFO)

# Silence the qiskit logger by setting its level to WARNING or higher
qiskit_logger = logging.getLogger("qiskit")
qiskit_logger.setLevel(logging.WARNING)

benchmark = Benchmark(
    [Trivial_Basic(coupling_map), SABRE(coupling_map)],
    [qasm_bench],
    [depth_metric],
    transpile_benchy_logger,
)
benchmark.run()

INFO:transpile_benchy:Running benchmarks for circuits...
Running circuits for QASMBench: 100%|██████████| 1/1 [00:00<00:00,  8.73it/s]


In [52]:
benchmark.plot(save=True)